In [ ]:
from ecmwf.opendata import Client
import numpy as np
import pandas as pd
import geopandas as gp
import requests
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
from datetime import datetime, timedelta
from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
import matplotlib.ticker as ticker
import shapely.speedups
from shapely import Polygon
import subprocess
from dataclasses import dataclass
import xarray as xr
import cfgrib
import zipfile
import os
import shutil
import rasterio
from rasterio.transform import from_origin
from rasterio.transform import from_bounds
from rasterio.crs import CRS
#import h3
from pyproj import Transformer
import json
import pprint
#import dask
import eccodes
import pygrib
import psycopg2
from osgeo import gdal
from collections import defaultdict
from scipy.spatial import KDTree
from itertools import chain

@dataclass
class DateTimeParts:
    year: int
    month: int
    day: int
    hour: int
    month_str: str
    day_str: str
    hour_str: str
    date_str: str

    @classmethod
    def from_datetime(cls, dt: datetime):
        month_str=str(dt.month).zfill(2)
        day_str = str(dt.day).zfill(2)
        hour_str=str(dt.hour).zfill(2)
        date_str = str(dt.year)+month_str+day_str
        return cls(year=dt.year, month=dt.month, day=dt.day, hour=dt.hour, month_str=month_str, day_str=day_str, hour_str=hour_str, date_str=date_str)
    
def windCalc(u,v):
        #print('windCalc Function')
        wind_abs = np.sqrt(u**2 + v**2)
        wind_dir_trig_to = np.arctan2(u/wind_abs, v/wind_abs)
        wind_dir_trig_to_degrees = wind_dir_trig_to * 180/np.pi ## -111.6 degrees
        wind_dir = wind_dir_trig_to_degrees + 180
        return wind_abs * 2.23694 #TO MPH
def K_to_F(temp):
    temp = ((temp - 273.15) * (9/5)) + 32
    return temp

def F_to_K(temp):
    temp = ((temp - 32) * 5/9) + 273.15
    return temp

In [ ]:
client = Client(source='aws')

latestrun = client.latest(
    #stream='enfo',
    step=[144],
    type="fc", #fc for HRES fcst #pf for perterbed fcst
    #param=["msl", "2t", "10u", "10v", "100u", "100v", "tp"],
    #param = ['mx2t6','mn2t6'],
    param = ['2t'],
)

latestrun.hour

In [ ]:
client = Client(source='ecmwf')
target_test = 'data/gribs/ecmwf/maxmin/ecmwf-latest_test.grib'

curtime = datetime.now()
if curtime.hour < 4:
    curtime = curtime - timedelta(days=1)
    day_str = f'{curtime.year}{str(curtime.month).zfill(2)}{str(curtime.day).zfill(2)}'
    hour = 12
    print(day_str, curtime.hour)
    step = list(range(12,144,3))
    client.retrieve(
    time=hour,
    date=day_str,
    #stream='enfo',
    step=step,
    type="fc", #fc for HRES fcst #pf for perterbed fcst
    #param=["msl", "2t", "tp"],
    param=["10u", "10v", "100u", "100v"],
    #param = ['mx2t6','mn2t6'],
    #param = ['2t'],
    target=target_test,
    )

else:
    latest= client.latest(
        #stream='enfo',
        step=[144],
        type="fc", #fc for HRES fcst #pf for perterbed fcst
        #param=["msl", "2t", "10u", "10v", "100u", "100v", "tp"],
        #param = ['mx2t6','mn2t6'],
        param = ['2t'],
    )
    latest = latest.hour

    print(latest)

    ens_range = chain(range(12,144,3),range(150,361,6))
    if latest in [6,18]:
        ens_range = range(12,144,3)
    step = list(ens_range)
    client = Client(source='aws')
    client.retrieve(
        #stream='enfo',
        step=step,
        type="fc", #fc for HRES fcst #pf for perterbed fcst
        param=["msl", "2t", "tp"],
        param=["10u", "10v", "100u", "100v"],
        #param = ['mx2t6','mn2t6'],
        #param = ['2t'],
        target=target_test,
    )

In [ ]:
#step = list(range(12,361,6))
ens_range = chain(range(12,144,3),range(150,361,6))
ens_range_aifs = range(0,361,6)
step = list(ens_range_aifs)
target = 'data/gribs/ecmwf-aifs/maxmin/ecmwf-aifs-latest.grib'
target2 = '/usr/share/geoserver/data_dir/data/ecmwf/ecmwf_latest.grib'
client = Client(source='ecmwf')
#step=[360]
client.retrieve(
    time=12,
    date=20241120,
    model="aifs",
    #stream='enfo',
    step=step,
    type="fc", #fc for HRES fcst #pf for perterbed fcst
    param = ['2t'],
    target=target,
)

In [ ]:
model = 'ecmwf-aifs'
#gribfile = f'data/gribs/{model.lower()}/latest/{model.lower()}-024.grb2'
#latestGrb = f'data/gribs/{model.lower()}/latest/{model.lower()}-latest.grib'
latestGrb = f'data/gribs/{model.lower()}/maxmin/{model.lower()}-latest.grib'
gribfile = latestGrb
grbs = pygrib.open(gribfile)
for grb in grbs:
    print(grb)

In [ ]:
#gribfile = f'data/gribs/{model.lower()}/maxmin/{model.lower()}-240.grb2'
grbs = pygrib.open(gribfile)
try:
    tgrb_max = grbs.select(name = 'Maximum temperature')
except:
    tgrb_max = []
try:
    tgrb_min = grbs.select(name = 'Minimum temperature')
except:
    tgrb_min = []
tempgrbs_max = (tgrb_max)
for  tmp in tempgrbs_max:
    print(tmp)
tempgrbs_min = (tgrb_min)
for  tmp in tempgrbs_min:
    print(tmp)

In [ ]:
#tgrb = grbs.select(shortName = '2t')
tgrb = grbs.select(name = 'Maximum temperature')
#tgrb = grbs.select(paramId = 167) #t2 ens std
tsd = (tgrb[3])
tsd4 = (tgrb[4])
print(tsd)
print(tsd4)
print(dir(tsd))
print(tsd.validDate)
tsd_list = list(dir(tsd))
tsd_keys = (tsd.keys())
print('2 metre temperature:K (instant):lambert:heightAboveGround:level 2 m:fcst time 36 hrs:from 202410151900:ens std dev')
for key in tsd_list:
    try:
        print(key, tsd[key])
    except:
        print(key, ' does not exist')
print('****************************************************')
print('2 metre temperature:K (instant):lambert:heightAboveGround:level 2 m:fcst time 36 hrs:from 202410151900')
tsd4_list = list(dir(tsd4))
tsd4_keys = (tsd4.keys())
for key4 in tsd4_list:
    try:
        print(key4, tsd4[key4])
    except:
        print(key4, ' does not exist')


In [ ]:
grb1 = grbs[1]
print(grb1)
lats, lons = grb1.latlons()
# Flatten the lat/lon arrays to create a 2D list of points
grid_points = np.column_stack((lats.ravel(), lons.ravel()))

# Build a KDTree from the lat/lon grid
tree = KDTree(grid_points)

In [ ]:
def createOutput(var, nearest_indices, df_coords):   
    data = {icao: [] for icao in df_coords['ICAO']}
    grbs_all = pygrib.open(gribfile)
    #tgrb = grbs_all.select(shortName = '2t')
    tgrb = grbs.select(name=var)
    tempgrbs = tgrb

    # Loop through the GRIB messages to extract data for all variables and times
    for grb in tempgrbs:
        print(grb)
        var_name = grb.name
        valid_time = grb.validDate

        # Flatten the data grid to align with the grid points
        data_values = (K_to_F(grb.values.ravel()).round(0))

        # For each row in the DataFrame, get the value from the nearest grid point
        for (icao, idx) in zip(df_coords['ICAO'], nearest_indices):
            nearest_value = data_values[idx]

            # Collect the value, time, and variable for the given ICAO
            data[icao].append({
                'time': valid_time,
                'variable': var_name,
                'value': int(nearest_value),
                'lat': df_coords.loc[df_coords['ICAO'] == icao, 'lat'].values[0],
                'lon': df_coords.loc[df_coords['ICAO'] == icao, 'lon'].values[0],
                'city': df_coords.loc[df_coords['ICAO'] == icao, 'CITY'].values[0],  # Add city
                'state': df_coords.loc[df_coords['ICAO'] == icao, 'STATE'].values[0]  # Add state
            })

    # Create an empty list to store rows for the DataFrame
    trows = []

    # Flatten the data dictionary into the list of rows
    for icao, records in data.items():
        for record in records:
            # Append each record as a dictionary to the rows list
            trows.append({
                'ICAO': icao,
                'time': record['time'],
                'variable': record['variable'],
                'value': record['value'],
                'lat': record['lat'],
                'lon': record['lon'],
                'CITY': record['city'],  # Add city to DataFrame
                'STATE': record['state']  # Add state to DataFrame
            })
    print(trows)

    # Convert the list of rows into a DataFrame
    result_df = pd.DataFrame(trows)

    # The final DataFrame contains the time series for each ICAO code
    print(result_df)

    features = []
    for icao, records in data.items():
        # Extract lat/lon from one of the records (all will have the same lat/lon for the same ICAO)
        lat = records[0]['lat']
        lon = records[0]['lon']
        city = records[0]['city']  # Extract city
        state = records[0]['state']  # Extract state

        # Prepare time series for this ICAO
        time_series = [{
            'time': str(record['time']),
            'variable': str(record['variable']),
            'value': str(record['value'])
        } for record in records]

        # Create the GeoJSON feature for this ICAO with the time series
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [str(lon), str(lat)]
            },
            "properties": {
                "ICAO": icao,
                "CITY": city,  # Add city to GeoJSON
                "STATE": state,  # Add state to GeoJSON
                "time_series": time_series
            }
        }
        features.append(feature)

    # Create the final GeoJSON structure
    geojson_data = {
        "type": "FeatureCollection",
        "features": features
    }

    # Determine file name based on the variable type
    if var == 'Maximum temperature at 2 metres in the last 6 hours':
        var_out = 'maxtemp'
    elif var == 'Minimum temperature at 2 metres in the last 6 hours':
        var_out = 'mintemp'
    elif var == '2 metre temperature':
        var_out = 'temp'   
    else:
        var_out = ''
    
    # Export to GeoJSON file
    """ with open(f'output_data_{var_out}.geojson', 'w') as f:
        json.dump(geojson_data, f, indent=4) """
    
    # Export to GeoJSON file
    with open(f'/var/www/fapi/app/static/data/point/ecmwf/output_data_{var_out}.geojson', 'w') as f:
        json.dump(geojson_data, f, indent=4)


    print("GeoJSON with time series exported successfully!")


df_coords = pd.read_csv('stations.csv')
coordinates = df_coords[['lat', 'lon']].values
nearest_indices = [tree.query([lat, lon])[1] for lat, lon in coordinates]
#createOutput('Maximum temperature at 2 metres in the last 6 hours', nearest_indices, df_coords)
#createOutput('Minimum temperature at 2 metres in the last 6 hours', nearest_indices, df_coords)
createOutput('2 metre temperature', nearest_indices, df_coords)


In [ ]:
with open('output_data.geojson', 'r') as f:
    geojson_data = json.load(f)

# Print to check structure
#print(json.dumps(geojson_data, indent=4))
print(geojson_data['features'])
print(geojson_data['features'][-1]['properties'])

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('stations2.csv')

# Print the original DataFrame
print("Original DataFrame:")
print(df)

# Define the custom order for the REGION column
regions = ['SOUTH', 'CENTRAL', 'NORTH', 'WEST']

# Convert the 'REGION' column to a categorical type with the specified order
df['REGION'] = pd.Categorical(df['REGION'], categories=regions, ordered=True)

# Sort the DataFrame by 'REGION' (custom order) and 'CITY' (alphabetically)
df_sorted = df.sort_values(['REGION', 'CITY'])

# Print the sorted DataFrame
print("Sorted DataFrame by REGION and CITY:")
print(df_sorted)

In [ ]:
import os
import datetime as dt


print(os.getcwd())
print(dir(dt.datetime))
print(dir(dt.timedelta))

In [ ]:
mydate = (dt.datetime.now())
mydate = dt.datetime(2024,1,1,16)
dateparts = DateTimeParts.from_datetime(mydate)
print(dateparts)
print(dateparts.year)
print(dateparts.month_str)
print(dateparts.day_str)
print(dateparts.hour_str)
print(dateparts.date_str)

if dateparts.hour >= 2 and dateparts.hour <= 8:
    hour_str = '00'
elif dateparts.hour >= 9 and dateparts.hour <= 15:
    hour_str = '06'
elif dateparts.hour >= 16 and dateparts.hour <= 21:
    hour_str = '12'
elif dateparts.hour >= 22:
    hour_str = '18'
elif dateparts.hour < 2:
    hour_str = '18'
    dateparts = DateTimeParts.from_datetime(mydate - dt.timedelta(days=1))
else:
    print(f'warning: forecast hour {dateparts.hour} is not between 0 and 23')
    hour_str = None

print('gfs runtime: ', dateparts.date_str, hour_str)


In [ ]:
def getInitTime_GFS():
    mydate = (dt.datetime.now())
    dateparts = DateTimeParts.from_datetime(mydate)

    if dateparts.hour >= 2 and dateparts.hour <= 8:
        hour_str = '00'
    elif dateparts.hour >= 9 and dateparts.hour <= 15:
        hour_str = '06'
    elif dateparts.hour >= 16 and dateparts.hour <= 21:
        hour_str = '12'
    elif dateparts.hour >= 22:
        hour_str = '18'
    elif dateparts.hour < 2:
        hour_str = '18'
        dateparts = DateTimeParts.from_datetime(mydate - dt.timedelta(days=1))
    else:
        print(f'warning: forecast hour {dateparts.hour} is not between 0 and 23')
        hour_str = None

    return dateparts, hour_str

dateparts, hour_str = getInitTime_GFS()
print(dateparts, hour_str)

In [ ]:
d = dt.datetime.now()

filename = f'file {d}.txt'

# Write the formatted string to the new file
with open(filename, 'w') as file:
    file.write(str(d))

In [ ]:
import matplotlib.colors as mcolors

color_values = {
    -30: 'maroon',
    -20: 'teal',
    -10: 'lavender',
    0: 'white',
    10: 'fuchsia',
    20: 'purple',
    30: 'blue',
    40: 'aqua',
    50: 'green',
    60: 'yellow',
    70: 'orange',
    80: 'red',
    90: 'purple',
    100: 'white',
    110: 'pink',
    130: 'maroon'
}

# Create the normalized bounds (0 to 1) for the colormap
norm = mcolors.Normalize(vmin=min(color_values.keys()), vmax=max(color_values.keys()))

# Create a LinearSegmentedColormap
cmap = mcolors.LinearSegmentedColormap.from_list(
    'colormap_tempF',
    [(norm(value), color) for value, color in color_values.items()]
)

import matplotlib.colors as mcolors

color_values_wind = {
    0: 'blue',
    5: 'aqua',
    10: 'yellow',
    20: 'orange',
    30: 'red',
    40: 'pink',
    50: 'green',
    60: 'brown'
}

# Create the normalized bounds (0 to 1) for the colormap
norm_wind = mcolors.Normalize(vmin=min(color_values_wind.keys()), vmax=max(color_values_wind.keys()))

# Create a LinearSegmentedColormap
cmap_wind = mcolors.LinearSegmentedColormap.from_list(
    'colormap_windms',
    [(norm_wind(value), color) for value, color in color_values_wind.items()]
)

In [ ]:
testfile = "data/gribs/ecmwf/maxmin/ecmwf-latest.grib"

grbs = pygrib.open(testfile)
print(dir(grbs[1]))
for grb in grbs[1:2]:
    #print(grb)
    print(grb.shortName, grb.typeOfLevel, grb.level)
    stepType = grb.stepType
    step = grb.step
    fhour = grb.validDate
    
    filter_kwargs = {'stepType' : f'{stepType}', 'step': step}
    ds = xr.open_dataset(
        testfile,
        engine="cfgrib",
        backend_kwargs={"filter_by_keys": filter_kwargs})
    print(ds)

In [ ]:
import matplotlib.ticker as ticker

model = 'ecmwf-aifs'
cmap_name = 'colormap_tempF'
testfile = "data/gribs/ecmwf/maxmin/ecmwf-latest.grib"
testfile = f'data/gribs/{model}/maxmin/{model}-latest.grib'
fapi_root = "/var/www/fapi/app/"
grbs = pygrib.open(testfile)
for grb in grbs[14:15]:
    shortname = grb.shortName
    longname = grb.name
    validDate = grb.validDate
    analDate = grb.analDate
    fcstHour = grb.forecastTime
    level = grb.level
    levtype = grb.levtype
    typeOfLevel = grb.typeOfLevel
    unit = grb.units
    cfName = grb.cfVarName
    stepType = grb.stepType
    step = grb.step
    p_units = grb.parameterUnits
    print('shortname', shortname)
    print('longname', longname)
    print('stepType', stepType)
    print('step', step)
    print('unit', unit)
    print('p_unit', p_units)
    #step = timedelta(hours=fcstHour)
    filter_kwargs = {'stepType' : f'{stepType}', 'step': step}
    #filter_kwargs = {'shortName' : f'{shortname}', 'step': step }
    ds = xr.open_dataset(
        testfile,
        engine="cfgrib",
        backend_kwargs={"filter_by_keys": filter_kwargs})
    ds=ds.sel(latitude = slice(57,20), longitude= slice(-130,-65))
    
    #ds = ds.sel(step=step)
    #print(ds)
    variables = list(ds.data_vars)
    variable = ds[variables[0]]
    variable = variable.rio.write_crs("EPSG:4326")
    variable.attrs['units'] = p_units
    variable = variable.metpy.quantify() 
    variable_F = variable.metpy.convert_units('degF')
    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    #contour = variable_F.plot(ax=ax, transform=ccrs.PlateCarree(), vmin=-30, vmax=130, cmap=cmap)
    #contour = ax.contourf(variable.longitude, variable.latitude, variable_F.values, transform=ccrs.PlateCarree(), levels = 32, vmin=-30, vmax=130, cmap=cmap)
    #contour = ax.pcolormesh(variable.longitude, variable.latitude, variable_F.values, transform=ccrs.PlateCarree(), vmin=-30, vmax=130, cmap=cmap, shading='gouraud')
    contour = ax.contourf(variable.longitude, variable.latitude, variable_F.values, transform=ccrs.PlateCarree(), levels=161, vmin=-30, vmax=130, cmap=cmap)
    ax.add_feature(cfeature.STATES)
    ax.coastlines()
    cbar = fig.colorbar(contour, shrink=1.0, orientation='horizontal', pad=0.03, aspect=60)
    cbar.ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    #colorbar = contour.colorbar
    #colorbar.ax.set_position([0.78, 0.3, 0.03, 0.4])
    #ax.set_extent([-130, -65, 20, 50])
    time_init = pd.Timestamp(ds.time.values).strftime('%D %H')
    time_valid = pd.Timestamp(ds.valid_time.values).strftime('%a %D %H')
    #plt.title(f"ECMWF {level}m wind mph {time_init}Z fcst hr {step} {time_valid}Z")
    extents = {
        'conus': [-130, -65, 20, 50],       # Global extent
        'central': [-105, -85, 28, 50],     # Regional extent
        'sc_plains': [-103, -95, 33, 39]          # Another regional extent
        }
    extents2 = {
        'central': [-103, -95, 33, 39]
        }
        #ax.set_extent([-130, -65, 20, 50])
        
    for region, extent in extents.items():
        ax.set_extent(extent)
        plt.title(f"{model.upper()} {shortname} deg F {time_init}Z fcst hr {step} {time_valid}Z")
        fapi_filename = f"static/data/plots/{model}/{model}_{region}_{shortname}_{str(step).zfill(3)}.png"
        fapi_path = f"{fapi_root}{fapi_filename}"
        plt.savefig(f"data/images/{model}_plot/{model}_{region}_{shortname}_{str(step).zfill(3)}.png", bbox_inches='tight', pad_inches=0.1)
        plt.savefig(fapi_path, bbox_inches='tight', pad_inches=0.1)
        #plt.savefig(f"data/images/{model}_plot/{model}_{shortname}_{str(step).zfill(3)}.png", bbox_inches='tight', pad_inches=0.1)

In [ ]:
testfile = "data/gribs/ecmwf/maxmin/ecmwf-latest_test.grib"
testfile = target
step_list = []
grbs = pygrib.open(testfile)
for grb in grbs:
    step_list.append(grb.step)
step_list = sorted(step_list)
steps = list(dict.fromkeys(step_list))
print(steps)

In [ ]:
testfile = "data/gribs/ecmwf/maxmin/ecmwf-latest_test.grib"
grbs = pygrib.open(testfile)
for step in list(steps)[1:3]:
    for level in [10,100]:
        print(step)
        grb = grbs.select(step=step, level=level)
        print(dir(grb[0]))
        shortname = grb.shortName
        print(shortname)
        filter_kwargs = {'level': level, 'step':step}
        ds = xr.open_dataset(
            testfile,
            engine="cfgrib",
            backend_kwargs={"filter_by_keys": filter_kwargs})
        print(ds)

In [ ]:
gdf_wind = gp.read_file('windfarms1000.shp')
print(gdf_wind)

In [ ]:


testfile = "data/gribs/ecmwf/maxmin/ecmwf-latest_test.grib"
grbs = pygrib.open(testfile)
for step in list(steps)[1:2]:
    for level in [10,100]:
        grb = grbs.select(step=step, level=level)[0]
        print(grb)
        filter_kwargs = {'level': level, 'step':step}
        print(dir())
        shortname = grb.shortName
        longname = grb.name
        validDate = grb.validDate
        analDate = grb.analDate
        fcstHour = grb.forecastTime
        level = grb.level
        levtype = grb.levtype
        typeOfLevel = grb.typeOfLevel
        unit = grb.units
        cfName = grb.cfVarName
        stepType = grb.stepType
        step = grb.step
        p_units = grb.parameterUnits
        print('shortname', shortname)
        print('longname', longname)
        print('stepType', stepType)
        print('timeinit',analDate)
        print('step', step)
        print('unit', unit)
        print('p_unit', p_units)
        #step = timedelta(hours=fcstHour)
        #filter_kwargs = {'stepType' : f'{stepType}', 'step': step}
        #filter_kwargs = {'shortName' : f'{shortname}', 'step': step }
        ds = xr.open_dataset(
            testfile,
        engine="cfgrib",
        backend_kwargs={"filter_by_keys": filter_kwargs})
        ds1=(ds.sel(latitude = slice(62,15), longitude= slice(-140,-55))).coarsen(latitude=2, longitude=2, boundary='trim').mean()
        ds=ds.sel(latitude = slice(57,20), longitude= slice(-130,-65))
        #ds = ds.sel(step=step)
        #print(ds)
        wind_u = f'u{level}'
        wind_v = f'v{level}'
        variable = windCalc(ds[wind_u].values, ds[wind_v].values)
        #variables = list(ds.data_vars)
        #variable = ds[variables[0]]
        #variable = variable.rio.write_crs("EPSG:4326")
        #variable.attrs['units'] = p_units
        #variable = variable.metpy.quantify() 
        #variable_F = variable.metpy.convert_units('degF')
        fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
        #contour = variable_F.plot(ax=ax, transform=ccrs.PlateCarree(), vmin=-30, vmax=130, cmap=cmap)
        #contour = ax.contourf(variable.longitude, variable.latitude, variable_F.values, transform=ccrs.PlateCarree(), levels = 32, vmin=-30, vmax=130, cmap=cmap)
        #contour = ax.pcolormesh(ds.longitude, ds.latitude, variable, transform=ccrs.PlateCarree(), vmin=0, vmax=60, cmap=cmap_wind, shading='gouraud')
        contour = ax.contourf(ds.longitude, ds.latitude, variable, transform=ccrs.PlateCarree(), levels=61, vmin=0, vmax=60, cmap=cmap_wind)
        gdf_wind.plot(ax=ax, color='black', edgecolor='black', markersize=0.4, alpha=1.0, label="wind farms")
        #ds1 = ds1.coarsen(latitude=2, longitude=2, boundary='trim').mean()
        #y, x = np.meshgrid(ds1.longitude, ds1.latitude)
        #ax.streamplot(y, x, ds1[wind_u].values, ds1[wind_v].values, density=[0.8, 1])
        ax.add_feature(cfeature.STATES)
        ax.coastlines()
        cbar = fig.colorbar(contour, shrink=1.0, orientation='horizontal', pad=0.03, aspect=60)
        cbar.ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
        #colorbar = contour.colorbar
        #colorbar.ax.set_position([0.78, 0.3, 0.03, 0.4])
        time_init = pd.Timestamp(ds.time.values).strftime('%D %H')
        time_valid = pd.Timestamp(ds.valid_time.values).strftime('%a %D %H')
        plt.title(f"ECMWF {level}m wind mph {time_init}Z fcst hr {step} {time_valid}Z")
        extents = {
            'conus': [-130, -65, 20, 50],       # Global extent
            'central': [-105, -85, 28, 50],     # Regional extent
            'sc_plains': [-103, -95, 33, 39]          # Another regional extent
            }
        extents2 = {
            'sc_plains': [-103, -95, 33, 39]
            }
        #ax.set_extent([-130, -65, 20, 50])
        
        for extent_version, extent in extents2.items():
            ax.set_extent(extent)
            plt.savefig(f"data/images/ecmwf_plot/ecmwf_{extent_version}_{shortname}_{str(step).zfill(3)}.png", bbox_inches='tight', pad_inches=0.1)

In [ ]:
import os
import psycopg2
from osgeo import gdal

# Database connection details
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'geoserver_mosaic'
DB_USER = 'geoserver_user'
DB_PASS = 'geoserver'


# Connect to the PostGIS database
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASS
)

# Create a cursor object
cur = conn.cursor()

def insert_metadata(file_path):
    # Extract filename
    filename = os.path.basename(file_path)
    
    # Extract date and time from filename (assuming format hrrr_wind-YYYYMMDDTHHmm.tiff)
    date_time_str = filename.split('-')[1].split('.')[0]  # Extract '20240915T1400'
    ingestion = f"{date_time_str[:4]}-{date_time_str[4:6]}-{date_time_str[6:8]} {date_time_str[8:10]}:00:00"
    
    # Open the TIFF file to get the bounding box (envelope)
    ds = gdal.Open(file_path)
    gt = ds.GetGeoTransform()
    minx = gt[0]
    maxx = gt[0] + (ds.RasterXSize * gt[1])
    miny = gt[3] + (ds.RasterYSize * gt[5])
    maxy = gt[3]

    # Delete old records with the same date and hour
    delete_query = "DELETE FROM public.hrrr_refd WHERE ingestion = %s"
    cur.execute(delete_query, (ingestion,))
    print(f"Deleted old records for {ingestion}")

    # Insert metadata into PostGIS table
    insert_query = """
        INSERT INTO public.hrrr_refd (location, ingestion, the_geom)
        VALUES (%s, %s, ST_MakeEnvelope(%s, %s, %s, %s, 3857))
    """

    #insert_query = "INSERT INTO IM_hrrr_refd.hrrr_refd (location, ingestion, the_geom) VALUES (%s, %s, ST_MakeEnvelope(%s, %s, %s, %s, 3857))"


    cur.execute(insert_query, (filename, ingestion, minx, miny, maxx, maxy))
    print(f"Inserted metadata for {filename}")

In [ ]:
import os
import psycopg2
from osgeo import gdal

def pg_conn():
    # Database connection details
    DB_HOST = 'localhost'
    DB_PORT = '5432'
    DB_NAME = 'geoserver_db'
    DB_USER = 'geoserver_user'
    DB_PASS = 'geoserver'

    # Connect to the PostGIS database
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASS
    )

    # Create a cursor object
    cur = conn.cursor()
    return conn, cur

In [ ]:

conn, cur = pg_conn()
model = 'ecmwf'
testfile = "data/gribs/ecmwf/maxmin/ecmwf-latest.grib"
grbs = pygrib.open(testfile)
for grb in grbs[1:9]:
    shortname = grb.shortName
    longname = grb.name
    validDate = grb.validDate
    analDate = grb.analDate
    fcstHour = grb.forecastTime
    level = grb.level
    levtype = grb.levtype
    typeOfLevel = grb.typeOfLevel
    unit = grb.units
    cfName = grb.cfVarName
    stepType = grb.stepType
    step = grb.step
    p_units = grb.parameterUnits
    ens_type = ''
    creation_time = datetime.now()
    fapi_root = "/var/www/fapi/app/"
    fapi_filename = f"static/data/images/{model}/{model}_{shortname}_{str(step).zfill(3)}.png"
    fapi_path = f"{fapi_root}{fapi_filename}"
    print('shortname', shortname)
    print('longname', longname)
    print('initTime', analDate)
    print('validDate', validDate)
    print('forecastime', fcstHour)
    print('stepType', stepType)
    print('step', step)
    print('creationTime', creation_time)

    insert_query = """
        INSERT INTO model_image.image_meta (path_rel, model, wxvar, "initTime", "validTime", step_hours, "stepType", level, ens_type, "creationTime")
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (model, wxvar, "initTime", "validTime", level, ens_type)
        DO NOTHING;
    """
    #print(insert_query, (fapi_filename, model, shortname, analDate, validDate, step, stepType, level, ens_type, creation_time))
    cur.execute(insert_query, (fapi_filename, model, shortname, analDate, validDate, step, stepType, level, ens_type, creation_time))
conn.commit()

In [1]:
%pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
